# Exercise 3: More tabular data wrangling

Skills:
* Looping
* Dictionary to map values
* Dealing with duplicates
* Make use of Markdown cells to write some narrative or commentary!

References:
* https://docs.calitp.org/data-infra/analytics_new_analysts/data-analysis-intermediate.html

In [1]:
import pandas as pd

In [2]:
GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/bus_service_increase/"
FILE_NAME = "ntd_metrics_2019.csv"

df = pd.read_csv(f"{GCS_FILE_PATH}{FILE_NAME}")
df.head(2)

,Agency,City,State,Legacy NTD ID,NTD ID,Organization Type,Reporter Type,Primary UZA\n Population,Agency VOMS,Mode,...,Passenger Miles Questionable,Vehicle Revenue Miles,Vehicle Revenue Miles Questionable,Any data questionable?,Unnamed: 39,Unnamed: 40,Unnamed: 41,1,Unnamed: 43,Unnamed: 44
0,MTA New York City Transit,New York,NY,2008,20008,"Subsidiary Unit of a Transit Agency, Reporting...",Full Reporter,"18,351,295","10,885",HR,...,NaN,"354,616,371",NaN,No,NaN,NaN,NaN,Hide questionable data tags,NaN,NaN
1,MTA New York City Transit,New York,NY,2008,20008,"Subsidiary Unit of a Transit Agency, Reporting...",Full Reporter,"18,351,295","10,885",CB,...,NaN,"9,866,807",NaN,No,NaN,NaN,NaN,Show questionable data tags,NaN,NaN


In [3]:
df.columns = df.columns.str.replace('\n', ' ').str.replace(' ', '_')

In [4]:
print(f"# obs: {len(df)}")
print(f"# unique IDs: {df.NTD_ID.nunique()}")

# obs: 3685
# unique IDs: 2183


In [5]:
# Pick an example -- see that agency provides service for different modes
# df.NTD_ID.value_counts()
df[df.NTD_ID=="10003"].Mode.value_counts()

MB    2
FB    1
DR    1
CR    1
RB    1
HR    1
TB    1
LR    1
Name: Mode, dtype: int64

### Dealing with Duplicates

* Explore why there are duplicates
* What's the analysis about? What should the unit of analysis be?
* Should duplicates be dropped? Should duplicates be aggregated into 1 entry?
* Hint: It depends on the analysis, and there might be a bit of both. Sometimes, aggregation makes sense. Duplicates require further investigation -- why do they appear in the dataset multiple times? Unless it's completely duplicate information, it doesn't make sense to just drop. It may show that the analysis can be more disaggregate than previously thought.

In [6]:
# But what about this case?
df[(df.Agency.str.contains("Los Angeles County Metropolitan Transportation Authority ")) 
   & (df.Agency.notna())].Mode.value_counts()

MB    2
HR    1
RB    1
LR    1
VP    1
Name: Mode, dtype: int64

In [7]:
# Find the column that has different values
df[(df.Agency.str.contains("Los Angeles County Metropolitan Transportation Authority ")) 
   & (df.Agency.notna()) & 
   (df.Mode=="MB")
  ]

,Agency,City,State,Legacy_NTD_ID,NTD_ID,Organization_Type,Reporter_Type,Primary_UZA__Population,Agency_VOMS,Mode,...,Passenger_Miles_Questionable,Vehicle_Revenue_Miles,Vehicle_Revenue_Miles_Questionable,Any_data_questionable?,Unnamed:_39,Unnamed:_40,Unnamed:_41,1,Unnamed:_43,Unnamed:_44
16,Los Angeles County Metropolitan Transportation...,Los Angeles,CA,9154,90154,Independent Public Agency or Authority of Tran...,Full Reporter,"12,150,996","3,469",MB,...,NaN,"65,595,822",NaN,No,NaN,NaN,NaN,NaN,NaN,NaN
17,Los Angeles County Metropolitan Transportation...,Los Angeles,CA,9154,90154,Independent Public Agency or Authority of Tran...,Full Reporter,"12,150,996","3,469",MB,...,NaN,"5,775,759",NaN,No,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
subset_cols = [
    'Agency', 'City', 'State', 'Legacy_NTD_ID', 'NTD_ID',
    'Organization_Type', 'Reporter_Type', 'Mode'
]

print(f"# obs: {len(df)}")
print(f"# obs after dropping dups: {len(df.drop_duplicates(subset=subset_cols))}")

# What does this indicate? Use Markdown cell and jot down some of the logic.

# obs: 3685
# obs after dropping dups: 3553


[Markdown reference](https://www.datacamp.com/community/tutorials/markdown-in-jupyter-notebook)

### Changing values by using a dictionary to map

In [ ]:
# Transit mode uses a code, 
# Use a dictionary to map those codes to its full name
MODE_NAMES = {
    'MB': 'Bus', 
    'LR': 'Light Rail',
    'CB': 'Commuter Bus',
}

# What happens to the ones that aren't specified in MODE_NAMES?
df = df.assign(
    mode_full_name = df.Mode.map(MODE_NAMES)
)

In [ ]:
df.mode_full_name.value_counts()

In [ ]:
df[df.mode_full_name.isna()].Mode.value_counts()

Map values from `Mode` to these categories: rail, bus, and other. 

Use `assign` and `map`.

### Looping

Can loop across columns or loop across subsets of data.

Sometimes, looping can make sense if you're repeating certain steps. Use it if it makes sense.

Here, for 2 different columns, `Agency_VOMS` and `Mode_VOMS`, the values show up as strings.

Print the dtypes out for all the columns. 

Print `value_counts()` for `Agency_VOMS` and `Mode_VOMS`. What is making these values appear as strings?

For those 2 columns, replace the commas with blanks and fill in missing values with `"0"` (zero, but with quotation marks to make it a string). 

Coerce these columns to be numeric.

In [ ]:
for c in ["Agency_VOMS", "Mode_VOMS"]:
    df[c] = df[c].str.replace(',', '').fillna('0').astype({c: int})

In [9]:
for s in ["CA", "ID"]:
    subset_df = df[df.State==s]
    display(subset_df[["Agency", "City"]].drop_duplicates().head())

,Agency,City
13,Los Angeles County Metropolitan Transportation...,Los Angeles
72,Orange County Transportation Authority,Orange
94,Access Services,El Monte
120,"City and County of San Francisco, dba: San Fra...",San Francisco
131,San Diego Metropolitan Transit System,San Diego


,Agency,City
703,"Ada County Highway District, dba: ACHD Commute...",Boise
778,Valley Regional Transit,Meridian
1440,"City of Pocatello, dba: Pocatello Regional Tra...",Pocatello
1482,Mountain Rides Transportation Authority,Ketchum
1598,Treasure Valley Transit,Nampa


### To Do:
* Keep a subset of columns and clean up column names (no spaces, newlines, etc):
    * columns related to identifying the agency
    * population, passenger trips
    * transit mode
    * at least 3 service metric variables, normalized and not normalized
* Deal with duplicates - what is the unit for each row? What is the unit for desired analysis? Should an agency appear multiple times, and if so, why?
* Aggregate at least 2 ways and show an interesting comparison, after dealing with duplicates somehow (either aggregation and/or defining what the unit of analysis is)
* Calculate weighted average after the aggregation for the service metrics
* Decide on one type of chart to visualize, and generalize it as a function
* Make charts using the function


### Step by Step

These are the 3 service metrics columns to keep (in addition to the columns listed above):
1. Fare Revenues  
1. Total Operating Expenses 
1. Vehicle Revenue Miles

The normalized columns are the ones adjusted by population or volume. 
* Instead of total fare revenues, it's the fare revenues per unlinked trip.
* Instead of total cost, it's cost per passenger or cost per hour.

Deal with duplicates. 

For an agency with multiple modes, aggregate it across modes and get the sum for the service metrics.

Ex: sum up the total fare revenues for an agency with rail, bus, and ferry modes. sum across the modes.

Does it make sense to sum up the normalized metrics?

For an agency with 3 modes (rail, bus, ferry) it make sense to sum up `fares_per_passenger` across those 3 modes? Why or why not?

If bus passengers make up 80% of the agency's passengers (rail 15%, ferry 5%), how do we make sure the normalized metric accounts for this? Bus fares are significantly lower than rail and ferry fares, in this scenario. How do we make sure that `fares_per_passenger` metric reflects this mix?

What is the correct way to calculate `fares_per_passenger` across modes for the same operator?

Show the correct way. Drop the existing normalized metrics and calculate it across modes for the agency. The resulting dataframe should be 1 row for each agency, with the service metrics aggregated to that agency across modes, as well as normalized metrics(per passenger or per passenger trip) across modes.

Make a bar chart for one service metric for 5 agencies (show both normalized and not normalized).

Ex: if you choose fare revenues, make a bar chart for total fare revenues and fare revenues per passenger trip. The 5 agencies should appear together on a single bar chart.


### Helpful Hints for Functions
* Opportunities are from components that are generalizable in making a chart
* Maybe these components need the same lines of code to clean them
* You can always further define variables within a function
* You can always use f-strings within functions to make slight modifications to the parameters you pass

In [ ]:
# Sample function
import altair as alt

def make_bar_chart(df, x_col, y_col):
    x_title = f"{x_col.title()}"
    
    chart = (alt.Chart(df)
             .mark_bar()
             .encode(
                 x=alt.X(x_col, title=x_title),
                 y=alt.Y(y_col, title=""),
             )
            )
    return chart
